In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [138]:
#loading data and dropping high cardinality object columns
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data = train_data.drop(['Cabin', 'Ticket', 'Name'], axis = 1)
test_data = test_data.drop(['Cabin', 'Ticket', 'Name'], axis = 1)

In [139]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#processing the data

my_imputer = SimpleImputer(strategy='most_frequent')

cat_var = (test_data.dtypes == 'object')
object_cols = list(cat_var[cat_var].index)

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

im_train_data = pd.DataFrame(my_imputer.fit_transform(train_data))
im_test_data = pd.DataFrame(my_imputer.fit_transform(test_data))

im_test_data.columns = test_data.columns
im_train_data.columns = train_data.columns

OH_cols_encoder_train = pd.DataFrame(OH_encoder.fit_transform(im_train_data[object_cols]))
OH_cols_encoder_test = pd.DataFrame(OH_encoder.fit_transform(im_test_data[object_cols]))

OH_cols_encoder_test.index = im_test_data.index
OH_cols_encoder_train.index = im_train_data.index

im_train_data = im_train_data.drop(object_cols, axis = 1)
im_test_data = im_test_data.drop(object_cols, axis = 1)

final_train_data = pd.concat([im_train_data, OH_cols_encoder_train], axis = 1)
final_test_data = pd.concat([im_test_data, OH_cols_encoder_test], axis = 1)

nn = ['PassengerId','Pclass','Age','SibSp','Parch','Fare']

for n in nn:
    final_train_data[n] = pd.to_numeric(final_train_data[n])
    final_test_data[n] = pd.to_numeric(final_test_data[n])
print('OK')

OK


In [140]:
features = ['PassengerId','Pclass','Age','SibSp','Parch','Fare', 0, 1, 2, 3, 4]

X = final_train_data[features]
y = final_train_data.Survived

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, random_state = 1)


In [149]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = i)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

pred = np.round(my_model.predict(X_valid)).astype(int)
mae = mean_absolute_error(pred, y_valid)
mae

[17:34:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.40782122905027934

In [153]:
final_model = XGBRegressor(n_estimator = 50)
final_model.fit(X,y,verbose=False)

prediction = np.round(final_model.predict(final_test_data[features])).astype(int)

filename = 'submission.csv'
pd.DataFrame({'PassengerId': final_test_data.PassengerId, 'Survived': prediction}).to_csv(filename, index=False)
prediction

[17:35:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,